In [1]:
source extend_bashkernel-2modes.source
set-global-evaluation-mode "my-script"

Ready.
Changing global mode to 'my-script'


In [ ]:
set-global-evaluation-mode "my-script"

# Yum repository

## Task: Install packages

### Task details

To create a and publish a yum repositry we need to install the ```createrepo``` and ```httpd``` commands.

### Example

```sudo yum install -y package```

#### Provided script

In [ ]:
# Task: yum-host-install-packages
# Evaluation Mode: provided-script

# Install packages

ssh -qi ../mykeypair root@10.0.2.100 'sudo yum install -y createrepo httpd'

### Check

In [ ]:
# Task: yum-host-install-packages
# Evaluation Mode: check

## Task: Start the httpd and configure startup setting

### Task details

Installing the package does not start up the service, it will have to be done manually. Running the service should generate the following output

```
Starting httpd: httpd: apr_sockaddr_info_get() failed for vagrant-centos6
httpd: Could not reliably determine the server's fully qualified domain name, using 127.0.0.1 for ServerName
                                                           [  OK  ]
```

We will also want to configure the server to start automatically at system startup. This can be done with the ```chkconfig``` command. The desired output should look like this:
```
httpd           0:off   1:off   2:on    3:on    4:on    5:on    6:off
```
Where ```3:on``` indicates that the service will run when the system boots up the next time


### Example

To start a service:
```
service service-name start
```

Manage services:
```
chkconfig service-name on|off
```

Confirm the status of services:
```
chkconfig --list service-name
```

#### Provided script

In [ ]:
# Task: yum-host-setup-httpd
# Evaluaition Mode: provided-script

# Configure and confirm/reconfirm httpd status

ssh -qi ../mykeypair root@10.0.2.100 <<'EOF'
sudo service httpd start
sudo chkconfig --list httpd
sudo chkconfig httpd on
sudo chkconfig --list httpd
EOF

### Check

In [ ]:
# Task: yum-host-setup-httpd
# Evaluation Mode: check

## Task: Setup the public folder for httpd

### Task details

This example will publish the folder /pub/. Create the folder
```
/var/www/html/pub
```
Then confirm that it was created.

### Example

```mkdir folder-name```

#### Provided script

In [ ]:
# Task: yum-host-setup-publish-folder
# Evaluation Mode: prodided-script

# Cretae the public folder httpd server and confirm

ssh -qi ../mykeypair root@10.0.2.100 <<'EOF'
sudo mkdir -p /var/www/html/pub
sudo ls -la /var/www/html/pub
EOF

### Check

In [ ]:
# Task: yum-host-setup-publish-folder
# Evaluation Mode: check

## Task: Confirm rpm file

### Task details

Make sure that the rpm file from the previous notebook exists.

### Example

```ls -la /path/to/file/name```

#### Provided script

In [ ]:
# Task: yum-host-confirm-rpm
# Evaluation Mode: provided-script

# Confirm that the rpm exists

ssh -qi ../mykeypair root@10.0.2.100 ' ls -l ${HOME}/rpmbuild/RPMS/x86_64/example-0.1.0-1.el6.x86_64.rpm'

### Check

In [ ]:
# Task: yum-host-confirm-rpm
# Evaluation Mode: check

## Task: Publish the rpm

### Task details

To publish the rpm we will make an identical copy of it inside the published folder. This example will make use of ```rsync``` to synchronize the rpm folder with the folder on out httpd server.

Go into the RPM folder inside the rpm and synchronize it with /var/www/html/pub/ using ```rsync```

### Example

```rsync -avx /from/ /to/```

#### Provided script

In [ ]:
# Task: yum-host-publish-rpm
# Evaluation Mode: provided-script

# Copy the rpm to newly created published folder

ssh -qi ../mykeypair root@10.0.2.100 <<'EOF'
cd ${HOME}/rpmbuild/RPMS/
sudo rsync -avx . /var/www/html/pub/
ls -l /var/www/html/pub/x86_64/example-0.1.0-1.el6.x86_64.rpm
EOF

### Check

In [ ]:
# Task: yum-host-publish-rpm
# Evaluation Mode: check

## Task: Create repo

### Task details

Inside the published folder use the ```createrepo``` command to create the repository. It should generate this output.
```
Spawning worker 0 with 1 pkgs
Workers Finished
Gathering worker results

Saving Primary metadata
Saving file lists metadata
Saving other metadata
Generating sqlite DBs
Sqlite DBs complete
```

It can be confirmed by looking inside the folder reopdata.

### Example

```createrepo```

#### Provided script

In [ ]:
# Task: yum-host-publish-repo
# Evaluation Mode: provided-script

# Create the repo

ssh -qi ../mykeypair root@10.0.2.100 <<'EOF'
cd /var/www/html/pub
sudo createrepo .
ls -l repodata/
EOF

### Check

In [ ]:
# Task: yum-host-publish-repo
# Evaluation Mode: check

## Task: Add to repository list

### Task details

Now to use the repositry a repo file is needed. The repo files aare located in the ```/etc/yum.repos.d/``` directory. This time we are creating a repo for ```example```, the repo file will therefore be called example.repo. The contents should look like this:
```
[example]
name=example
baseurl=http://127.0.0.1/pub/
enabled=1
gpgcheck=0
```

Where baseurl specifies where the repo is to be downloaded from. (This time we are using the locally published ```/pub/``` folder)

### Example

```EXAMPLE_CODE```

#### Provided script

In [ ]:
# Task: yum-host-add-repo
# Evaluation Mode: provided-script

# Add the repo to list

ssh -qi ../mykeypair root@10.0.2.100 <<'EOS'
cat <<EOF > /etc/yum.repos.d/example.repo
[example]
name=example
baseurl=http://127.0.0.1/pub/
enabled=1
gpgcheck=0
EOF
EOS

### Check

In [ ]:
# Task: yum-host-add-repo
# Evaluation Mode: check

## Task: Confirm and install repo

### Task details

We can now run some commands to make sure that the repository is available and also get information about the package.
```
sudo yum makecache --disablerepo='*' --enablerepo='example'
```
Should give us this
```
Loaded plugins: fastestmirror
Loading mirror speeds from cached hostfile
example                                                  | 2.9 kB     00:00
example/filelists_db                                     |  748 B     00:00
example/other_db                                         |  650 B     00:00
Metadata Cache Created
```
This confirms that the example is available
```
yum info example
```
Should output this
```
Loaded plugins: fastestmirror
Determining fastest mirrors
 * base: www.ftp.ne.jp
 * extras: www.ftp.ne.jp
 * updates: www.ftp.ne.jp
Available Packages
Name        : example
Arch        : x86_64
Version     : 0.1.0
Release     : 1.el6
Size        : 1.7 k
Repo        : example
Summary     : example
License     : BSD
Description :
```
This gives us some information about the package

Now install the package using ```yum```

### Example

```yum install -y package-name```

#### Provided script

In [ ]:
# Task: yum-host-install-example-package
# Evaluation Mode: provided-script

# Check information and install the example package

ssh -qi ../mykeypair root@10.0.2.100 <<'EOF'
sudo yum makecache --disablerepo='*' --enablerepo='example
sudo yum info example
sudo yum install -y example

### Check

In [ ]:
# Task: yum-host-install-example-package
# Evaluation Mode: check

## Task: Upgrade package

### Task details

Now that we have created a repository we will also manage the repository for an upgraded version, example-0.2.0.
Go into the SOURCES folder of our rpm and make a copy of example-0.1.0 name it example-0.2.0 using ```rsync```
And like in the previous excersise make it into an archive using ```tar```

### Example

Refer to previous examples

#### Provided script

In [ ]:
# Task: yum-host-upgrade-version
# Evaluation Mode: provided-script

# Adding a new version

ssh -qi ../mykeypair root@10.0.2.100 <<'EOF'
cd ${HOME}/rpmbuild/SOURCES
rsync -avx example-0.1.0/ example-0.2.0
tar zcvf example-0.2.0.tar.gz example-0.2.0
EOF

### Check

In [ ]:
# Task: yum-host-upgrade-version
# Evaluation Mode: check

## Task: Upgrade the spec file

### Task details

The spec file also needs to be updated to indicate that example is now version 0.2.0.
Go into the SPECS folder, make a backup of example.spec. Then edit the file that is being used and replace 0.1.0 with 0.2.0 for the version.

### Example

```EXAMPLE_CODE```

#### Provided script

In [ ]:
# Task: yum-host-upgrade-spec
# Evaluation Mode: provided-script

# Upgrade the information inside spec file

ssh -qi ../mykeypair root@10.0.2.100 <<'EOF'
cd ${HOME}/rpmbuild/SPECS/
cp -pi example.spec example.spec.orig
sed -i 's,0.1.0,0.2.0,' example.spec
diff example.spec.orig example.spec
EOF

### Check

In [ ]:
# Task: yum-host-upgrade-spec
# Evaluation Mode: check

## Task: Rebuild rpm file

### Task details

Build the spec again, using the ```rpmbuild``` command. The same way we used it in the previous exercise.

### Example

Refer to 107

#### Provided script

In [ ]:
# Task: yum-host-upgrade-rebuild-spec
# Evaluation Mode: provided-script

# Rebuild the spec file to be versoin 0.2.0

ssh -qi ../mykeypair root@10.0.2.100 <<'EOF'
cd ${HOME}/rpmbuild/SPECS/
rpmbuild -bb example.spec
EOF

### Check

In [ ]:
# Task: yum-host-upgrade-rebuild-spec
# Evaluation Mode: check

## Task: Resync /var/www/html/pub/

### Task details

Because the version was updated the /pub/ folder needs to be updated as well.

Use ```rsync``` to update the folder to the new version from the RPMS folder.

### Example

Refer to previous example on how to publish rpm to repository

#### Provided script

In [ ]:
# Task: yum-host-upgrade-republish-spec
# Evaluation Mode: provided-script

# Publish new version on the webserver

ssh -qi ../mykeypair root@10.0.2.100 <<'EOF'
cd ${HOME}/rpmbuild/RPMS/
sudo rsync -avx . /var/www/html/pub
ls -l /var/www/html/pub/x86_64/example-0.2.0-1.el6.x86_64.
EOF

### Check

In [ ]:
# Task: yum-host-upgrade-republish-spec
# Evaluation Mode: check

## Task: Recreate repo

### Task details

Go to ```/var/www/html/pub/``` and create the repo once again.

### Example

Refer to previous example

#### Provided script

In [ ]:
# Task: yum-host-upgrade-publish-repo

ssh -qi ../mykeypair root@10.0.2.100 <<'EOF'
cd /var/www/html/pub
sudo createrepo .
EOF

### Check

In [ ]:
# Task: yum-host-upgrade-publish-repo
# Evaluation Mode: check

## Task:

### Task details

DECRPTION

### Example

```EXAMPLE_CODE```

#### Provided script

In [ ]:
# Task: yum-host-upgrade-reset-metadata
ssh -qi ../mykeypair root@10.0.2.100 <<'EOF'
yum clean metadata
yum info example
EOF

### Check

In [ ]:
# Task: yum-host-upgrade-reset-metadata
# Evaluation Mode: check

## Task:

### Task details

DECRPTION

### Example

```EXAMPLE_CODE```

#### Provided script

In [ ]:
# Task: yum-host-upgrade-install
ssh -qi ../mykeypair root@10.0.2.100 <<'EOF'
sudo yum update -y example
rpm -qi example
EOF

### Check

In [ ]:
# Task: yumhost-upgrade-install
# Evaluation Mode: check